In [1]:
import pandas as pd 
import numpy as np 
import datetime 
import matplotlib.pyplot as plt
import copy
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

In [2]:
business = pd.read_csv('business.csv')

/Users/srai/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (1,4,7,17,26,29,49,60,62,79,86,94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
business.columns

Index(['attributes.Ambience.divey', 'attributes.Dietary Restrictions.vegan',
       'attributes.Happy Hour', 'hours.Thursday.open',
       'attributes.Order at Counter',
       'attributes.Hair Types Specialized In.africanamerican',
       'attributes.Hair Types Specialized In.kids', 'attributes.BYOB',
       'hours.Friday.open', 'attributes.Good For.latenight',
       'attributes.Outdoor Seating', 'attributes.Alcohol',
       'attributes.Ambience.classy', 'attributes.By Appointment Only',
       'attributes.Parking.lot', 'business_id', 'attributes.Ambience.touristy',
       'attributes.Corkage', 'hours.Tuesday.open',
       'attributes.Good For.brunch', 'categories', 'attributes.Waiter Service',
       'hours.Monday.open', 'name', 'attributes.Parking.street',
       'attributes.Ambience.hipster', 'attributes.BYOB/Corkage',
       'attributes.Hair Types Specialized In.straightperms',
       'attributes.Music.live', 'attributes.Dietary Restrictions.dairy-free',
       'attributes.Music.

In [4]:
business.head()

,attributes.Ambience.divey,attributes.Dietary Restrictions.vegan,attributes.Happy Hour,hours.Thursday.open,attributes.Order at Counter,attributes.Hair Types Specialized In.africanamerican,attributes.Hair Types Specialized In.kids,attributes.BYOB,hours.Friday.open,attributes.Good For.latenight,...,attributes.Noise Level,attributes.Smoking,attributes.Attire,attributes.Hair Types Specialized In.curly,attributes.Good For Groups,neighborhoods,attributes.Open 24 Hours,attributes.Ambience.romantic,attributes.Music.jukebox,attributes.Ambience.upscale
0,False,NaN,NaN,11:00,NaN,NaN,NaN,NaN,11:00,False,...,average,NaN,casual,NaN,True,[],NaN,False,NaN,False
1,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,True,[],NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,True,NaN,False,10:00,NaN,NaN,NaN,NaN,10:00,False,...,average,no,casual,NaN,True,[],NaN,False,NaN,False
4,NaN,NaN,NaN,11:00,NaN,NaN,NaN,NaN,11:00,NaN,...,NaN,NaN,NaN,NaN,True,[],NaN,NaN,NaN,NaN


## Lets take a simple problem 
#### Can we predict the star rating of a business based on some very simple business attributes such as location . The motivation behind this is that, often we will find examples where a business has not been rated by users, (for example new business). In cases like this it will be useful if we can predict a star rating based on some basic atributes of the business.

In [5]:
X = business
y = business['stars']
X_train_index,X_test_index,y_train,y_test = train_test_split(X.index,y,test_size=0.2, random_state = 1)

In [6]:
numeric_columns = ['latitude', 'longitude']

In [7]:
X_train = X.iloc[X_train_index].as_matrix(numeric_columns)
X_test = X.iloc[X_test_index].as_matrix(numeric_columns)

In [8]:
import sklearn.linear_model
lr = sklearn.linear_model.LinearRegression()
model_lr = lr.fit(X_train, y_train)

In [9]:
model_lr.score(X_train, y_train)

0.00033301485390291319

In [10]:
model_lr.score(X_test, y_test)

0.00022648665467461804

#### Above scores suggest that Linear Regression model does not fit the data well, in fact it underfits. The training score is low, and the testing score is also low

#### Lets now add some non-numeric features to the model and see if the score improves. For doing this we need to identify non-numeric features which we beleive based on understanding of the problem will improve the prediction. As an example lets add all attributes of the business which provide information about the ambience, and also its price range. These attributes together with location might be good predictors for the star rating

In [11]:
non_numeric_columns_ambience = [k for k in business.columns if 'Ambience' in k]
feature_columns = ['latitude', 'longitude','attributes.Price Range']
feature_columns.extend(non_numeric_columns_ambience)
X = business
y = business['stars']
X_train_index,X_test_index,y_train,y_test = train_test_split(X.index,y,test_size=0.2, random_state = 1)
X_train = X.iloc[X_train_index].fillna(-1).as_matrix(feature_columns)
X_test = X.iloc[X_test_index].fillna(-1).as_matrix(feature_columns)
lr = sklearn.linear_model.LinearRegression()
model_lr = lr.fit(X_train, y_train)
print("Train Score = ", model_lr.score(X_train, y_train))
print("Test Score = ", model_lr.score(X_test, y_test))

Train Score =  0.0141083917798
Test Score =  0.0139446515319


####  Above attributes improve our R2 score very slightly for predicting the review count of a business based on its meta data attributes. Lets formalize this approach a bit, so that we can apply this more easily to other examples, and continue our search for better models to improve our scores

In [12]:
def get_columns_from_prefix(biz_df, prefix):
    return [col for col in biz_df.columns if prefix in col]
def eval_score(model, model_name, biz_df, numeric_col, other_col_prefix, y_col_name):
    X = biz_df
    y = biz_df[y_col_name]
    feature_columns = copy.copy(numeric_col)
    for col_prefix in other_col_prefix:
        feature_columns.extend(get_columns_from_prefix(biz_df, col_prefix))
    X_train_index,X_test_index,y_train,y_test = train_test_split(X.index,y,test_size=0.2, random_state = 1)    
    X_train = X.iloc[X_train_index].fillna(0).as_matrix(feature_columns)
    X_test = X.iloc[X_test_index].fillna(0).as_matrix(feature_columns)
    model = model.fit(X_train, y_train)
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)
    print('Model Name : {0}, Train Score = {1}, Test Score = {2}, Train Set size = {3}'.format(model_name, train_score,test_score,X_train.shape))
    return {
        'X_train':X_train,
        'X_test':X_test,
        'y_train':y_train,
        'y_test':y_test,
        'model':model,
        'train_score':train_score,
        'test_score':test_score}

#### Lets run the same example from before with above functions to confirm results produced are identical

In [13]:
numeric_col_names = ['latitude', 'longitude','attributes.Price Range']
other_col_prefix = ['Ambience']
y_col_name = 'stars'
model = sklearn.linear_model.LinearRegression()
eval_res = eval_score(model, 'LinearRegression', business, numeric_col_names, other_col_prefix, y_col_name)

Model Name : LinearRegression, Train Score = 0.006101224860929788, Test Score = 0.006819635864603879, Train Set size = (68720, 12)


#### Evaluation results are identical to previous run. Now lets add more features, and evaluate if it improves the model scores

In [14]:
numeric_col_names = ['latitude', 'longitude','attributes.Price Range']
other_col_prefix = ['Ambience','Good For','Music', 'Parking', 'Dietary Restrictions', 'Hair']
y_col_name = 'stars'
model = sklearn.linear_model.LinearRegression()
eval_res = eval_score(model,'LinearRegression', business, numeric_col_names, other_col_prefix, y_col_name)

Model Name : LinearRegression, Train Score = 0.036814226074976864, Test Score = 0.037212404973611224, Train Set size = (68720, 46)


#### Addition of more features improved the R2 score on test data slightly. Notice that we have not yet considered the category information while computing these models. Lets try adding the category infomration as well. 

#### Lets add all information available from categores as boolean variables. For example if a business has 'Mexican' as a category, we will add a columns to our data frame that will be True when 'Mexican' is a category present in the 'categories' column for that business. To compute this we will need to know the set of all categories first 

In [15]:
def add_categories(business):
    all_categories = pd.unique([item for sublist in business['categories'].apply(eval) for item in sublist])
    for cat in all_categories:
        business['computed.category.' + str(cat)] = business['categories'].apply(lambda x: True if cat in x else False)
    return business
business = add_categories(business)

#### Lets compute new models now that use these additional columns 

In [16]:
numeric_col_names = ['latitude', 'longitude','attributes.Price Range']
other_col_prefix = ['Ambience','Good For','Music', 'Parking', 'Dietary Restrictions', 'Hair', 'computed.category.']
y_col_name = 'stars'
lr = LinearRegression()
eval_res = eval_score(lr,'LinearRegression', business, numeric_col_names, other_col_prefix, y_col_name)

Model Name : LinearRegression, Train Score = 0.25138468773889755, Test Score = 0.22334746185294585, Train Set size = (68720, 1078)


#### The train score is  slightly greater than the test score,indicating overfitting in this linear regression model. Lets see if building a model that can capture the non-linearity and interactiion effects amongst the features. RandomForest and GradientBoosting approaches are both capable of acheiving this. We expect both the train and test scores to imrove as a result of using these models.   These models  takes a few minutes to generate

In [ ]:
rf = RandomForestRegressor(n_estimators=100, max_depth = 12, verbose = 1,n_jobs = 4)
eval_res = eval_score(rf,'Random Forest', business, numeric_col_names, other_col_prefix, y_col_name)

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   52.4s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  1.9min finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished


Model Name : Random Forest, Train Score = 0.21186252202353684, Test Score = 0.16871236196909656, Train Set size = (68720, 1078)


#### The random forest model above has lower score on both train data as well as test data. This suggests that the random forest as defined above is not able to fit the data in a manner comparable with Linear Regression

#### We continue to search for models that can capture the patterns in the data in a better manner. Lets continue to search for a model that improves the training and test scores beyond the Linear Regression model. Lets explore now if gradient boosting regression, another advanced regression technique can improve the fit by improving the score on our training, as well as test set 

In [ ]:
gbr = GradientBoostingRegressor(max_depth = 10,  verbose=10)
eval_res = eval_score(gbr, 'Gradient Boosting',business, numeric_col_names, other_col_prefix, y_col_name)

      Iter       Train Loss   Remaining Time 
         1           0.8674           10.46m
         2           0.8455           11.73m
         3           0.8274           11.36m
         4           0.8098           11.09m


#### Above Gradient Boosting model improves both the training score as well as test score in comparision with the LinearRegression model. However there is a significant gap between the train and test scores. This indicates overfitting. At this stage we expect that reducing model complexity should not hurt the test score.  Lets reduce model complexity by reducing depth of individual trees in the ensamble

In [ ]:
gbr = GradientBoostingRegressor(max_depth = 10,  verbose=10)
eval_res = eval_score(gbr, 'Gradient Boosting',business, numeric_col_names, other_col_prefix, y_col_name)

#### As expected the gap between the test and train scores has reduced. However the test score has also reduced.  
#### The best model for improving the scores on test data set so far, is the gradient boosted model with  individual trees in the ensamble grown to a depth of 10

#### Lets try to bring in features into the model that also include user behavior. 
#### Does the prediction of a business star rating improve if we know the number of checkins to the business on different days?

#### Lets get the check in data 

In [ ]:
checkin = pd.read_csv('checkin.csv')

In [ ]:
checkin.head()


#### Lets see if the prediction scores for star rating of a business can be improved if we also include the check in info as one of the predictors. Note that this attribute is a user interaction based attribute, and deviates slightly from our use case that a business was new. However this score  

In [ ]:
business_checkin = business.merge(checkin, on = 'business_id', how = 'left')

In [ ]:
business_checkin.head()

#### Lets add all the 'checkin_info' from the business_checkin data frame as features into the above model

In [ ]:
numeric_col_names = ['latitude', 'longitude','attributes.Price Range']
other_col_prefix = ['Ambience','Good For','Music', 'Parking', 'Dietary Restrictions', 'Hair', 'computed.category.', 'checkin_info']
y_col_name = 'stars'
lr = LinearRegression()
eval_res = eval_score(lr, 'LinearRegression', business_checkin, numeric_col_names, other_col_prefix, y_col_name)

#### Lets also evaluate the model scores when we train the gradient boosting configuration from previous examples that gave us better scores than other models

#### So far the R2 score of 0.25196846789989025 is the best we have for predicting a business's star rating using its meta data, and check in info. 

In [ ]:
eval_res['X_train_index']

#### Now lets change our problem definition slightly, to see if this problem can be solved as a classification problem. 
#### Lets try to predict now whether a business has a 'Good Score' or 'Bad Score' based on its meta data and check in info or not 
#### We define a 'Good Score' for business to be any rating >= average (computed over the training set)

In [ ]:
mean_star_rating = eval_res['y_train'].mean()

In [ ]:
mean_star_rating

In [ ]:
business_checkin['good_score'] = business_checkin['stars'].apply(lambda x: x >= mean_star_rating)

#### Lets train a logistic regression model on this, and create a classification model using only business meta data as its features (i.e not including checkin info)

In [ ]:
numeric_col_names = ['latitude', 'longitude','attributes.Price Range']
other_col_prefix = ['Ambience','Good For','Music', 'Parking', 'Dietary Restrictions', 'Hair', 'computed.category.']
y_col_name = 'good_score'
lr = LogisticRegression()
eval_res = eval_score(lr, 'LogisticRegression', business_checkin, numeric_col_names, other_col_prefix, y_col_name)

#### The scores above are high, but can be mis-leading, lets compute the precision recall curves 

In [ ]:
%matplotlib inline 
def plot_precision_recall(model, X_test, y_test):
    y_score = model.decision_function(X_test)
    precision, recall, threshold = precision_recall_curve(y_test,y_score)
    average_precision = average_precision_score(y_test, y_score)
    plt.clf()
    plt.plot(recall, precision, label='Precision-Recall curve')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('Precision-Recall example: AUC={0:0.2f}'.format(average_precision))
    plt.legend(loc="lower left")
    plt.show()
    return pd.DataFrame({'precision':precision[1:], 'recall':recall[1:], 'threshold':threshold})
precision_recall_threshold = plot_precision_recall(lr, eval_res['X_test'], eval_res['y_test'])    

#### An AUC of 0.74 as above, indicates that the model fits the data reasonably well, by choosing appropriate threshold for predicton we can achieve the desired precision, as well as recall

#### Any of the threshold values from above table will yeild a precision and recall > 0.68

In [ ]:
precision_recall_threshold.query ('precision > 0.68 and recall > 0.68').head()

In [ ]:
numeric_col_names = ['latitude', 'longitude','attributes.Price Range']
other_col_prefix = ['Ambience','Good For','Music', 'Parking', 'Dietary Restrictions', 'Hair', 'computed.category.','checkin_info']
y_col_name = 'good_score'
lr = LogisticRegression()
eval_res = eval_score(lr, 'LogisticRegression', business_checkin, numeric_col_names, other_col_prefix, y_col_name)

In [ ]:
precision_recall_threshold = plot_precision_recall(lr, eval_res['X_test'], eval_res['y_test'])  

In [ ]:
gbr = GradientBoostingClassifier(n_estimators=100, max_depth = 8,  verbose=10)
eval_res = eval_score(gbr, 'Gradient Boosting Classifier', business_checkin, numeric_col_names, other_col_prefix, y_col_name)

In [ ]:
precision_recall_threshold = plot_precision_recall(gbr, eval_res['X_test'], eval_res['y_test'])  

In [ ]:
precision_recall_threshold.query ('precision > 0.68 and recall > 0.68').head()

#### Computing Gradient boosted model for regression with a depth of 300
#### This takes a long time to generate, hence computing this at the end

In [ ]:
numeric_col_names = ['latitude', 'longitude','attributes.Price Range']
other_col_prefix = ['Ambience','Good For','Music', 'Parking', 'Dietary Restrictions', 'Hair', 'computed.category.', 'checkin_info']
y_col_name = 'stars'
lr = LinearRegression()
eval_res = eval_score(lr, 'LinearRegression', business_checkin, numeric_col_names, other_col_prefix, y_col_name)
gbr = GradientBoostingRegressor(n_estimators=300, max_depth = 8,  verbose=10)
eval_res = eval_score(gbr, 'Gradient Boosting', business_checkin, numeric_col_names, other_col_prefix, y_col_name)